# A Critique of the Bounded Fuzzy Possibilistic Method 
## Yeast Data Set

The *Bounded Fuzzy Possibilistic Method* (BFPM) was recently introduced in [doi:10.1016/j.fss.2019.07.011](https://dx.doi.org/10.1016/j.fss.2019.07.011). I demonstrate that there are some critical flaws in the proposed algorithm, which makes the results presented therein highly questionable. In particular, the method may generate meaningless cluster membership degrees or even fail to converge when run on some classical benchmark data sets.

In [1]:
import numpy as np
import pandas as pd
from bfpm import partition  # see bfpm.py in this repository — an implementation of the BFPM and Fuzzy c-means
np.random.seed(123)  # reproducibility matters

Let's consider the *Yeast* data set that can be downloaded from the UCI Machine Learning repository (https://archive.ics.uci.edu/ml/).

In [2]:
# https://archive.ics.uci.edu/ml/machine-learning-databases/iris/
# https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/
yeast = pd.read_csv("yeast.data", sep="\\s+",
    names=["name", "x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "y"])
display(yeast.head())  # preview

,name,x1,x2,x3,x4,x5,x6,x7,x8,y
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


The first column gives sequence names which are deemed irrelevant to our task and hence will be removed.

In [3]:
yeast = yeast.iloc[:,1:] # skip 1st column (sequence names)

Note that the dataset yields the following $n$ (number of input points) and $c$ (number of reference clusters), respectively:

In [4]:
yeast.shape[0], len(np.unique(yeast.y))

(1484, 10)

However, Table 1 in [doi:10.1016/j.fss.2019.07.011](https://dx.doi.org/10.1016/j.fss.2019.07.011) gives 1299 and 4, respectively.
Here is the contingency table for the reference class labels:

In [5]:
yeast.y.value_counts().sort_values().to_frame().T

,ERL,POX,VAC,EXC,ME1,ME2,ME3,MIT,NUC,CYT
y,5,20,30,35,44,51,163,244,429,463


It seems (but this is an educated guess only) that in [doi:10.1016/j.fss.2019.07.011](https://dx.doi.org/10.1016/j.fss.2019.07.011), only the classes `ME3`, `MIT`, `NUC`, and `CYT` were considered (which gives 1299 observations in total).
Therefore, from now on we shall restrict ourselves only to these 4 classes.

In [6]:
yeast2 = yeast.loc[yeast.y.isin(["ME3", "MIT", "NUC", "CYT"]),:]
yeast2_X = np.array(yeast2.iloc[:,:8])
yeast2_Y = np.array(yeast2.y.astype("category").cat.codes)
yeast2_c = len(np.unique(yeast2_Y)) # number of reference clusters
yeast2.shape[0], yeast2_c # n and c, respectively

(1299, 4)

The 4 reference cluster centres are as follows:

In [7]:
yeast2_V_true = np.array(yeast2.groupby("y").mean())
print(np.round(yeast2_V_true, 4))

[[0.4807 0.4695 0.5354 0.2271 0.5043 0.0011 0.4994 0.2572]
 [0.4309 0.4895 0.3643 0.2135 0.5031 0.     0.5101 0.27  ]
 [0.5214 0.5332 0.5174 0.4044 0.5    0.0089 0.5017 0.241 ]
 [0.4524 0.4561 0.5293 0.2283 0.5035 0.     0.4941 0.3317]]


Running the FCM with fuzzifier $m=2$ and random initial $V$ yields:

In [8]:
yeast2_V_FCM = partition(yeast2_X, yeast2_c,
    V_init=None, m=2.0, use_FCM=True)
print(np.round(yeast2_V_FCM, 4))

[[0.5203 0.5152 0.5052 0.214  0.5023 0.0015 0.5042 0.2695]
 [0.5061 0.5106 0.514  0.4544 0.5022 0.0018 0.4985 0.2597]
 [0.4315 0.4449 0.5104 0.216  0.5027 0.0016 0.4987 0.2864]
 [0.4435 0.4551 0.509  0.2177 0.5027 0.0016 0.4993 0.2853]]


The solution does not match the reference cluster centres well. Nevertheless, it is well-known that the FCM might decrease its performance in higher-dimensional spaces. Also, the above true cluster memberships may be an instance of possibly many valid ones.

On the other hand, running the BFPM ($m=2$, random initial $V$) gives:

In [9]:
yeast2_V_BFPM = partition(yeast2_X, yeast2_c,
    V_init=None, m=2.0, V_prev_history_size=1, use_FCM=False, verbose=True)
print(np.round(yeast2_V_BFPM, 4))

[[0.4728 0.4796 0.5085 0.2591 0.5031 0.002  0.4994 0.2804]
 [0.4728 0.4796 0.5085 0.2591 0.5031 0.002  0.4994 0.2804]
 [0.4728 0.4796 0.5085 0.2591 0.5031 0.002  0.4994 0.2804]
 [0.4728 0.4796 0.5085 0.2591 0.5031 0.002  0.4994 0.2804]]


The algorithm converged to a solution where
all cluster centres coincide and are equal to the centroid of the whole
data set. We obtain the same outcome if the BFPM is initialised from the true cluster centres.

In [10]:
yeast2_V_BFPM = partition(yeast2_X, yeast2_c,
    V_init=yeast2_V_true, m=2.0, V_prev_history_size=1, use_FCM=False, verbose=True)
print(np.round(yeast2_V_BFPM, 4))

[[0.4728 0.4796 0.5085 0.2591 0.5031 0.002  0.4994 0.2804]
 [0.4728 0.4796 0.5085 0.2591 0.5031 0.002  0.4994 0.2804]
 [0.4728 0.4796 0.5085 0.2591 0.5031 0.002  0.4994 0.2804]
 [0.4728 0.4796 0.5085 0.2591 0.5031 0.002  0.4994 0.2804]]
